In [ ]:
# for debugging
import sys, importlib
if 'api' in sys.modules:
    importlib.reload(sys.modules['api'])
    importlib.reload(sys.modules["json_parser"])

# real
from api import search_posts, get_comments
from csv_dumper import extract_post_data, extract_comment_data

In [84]:
posts_res = search_posts("Coldplay", "stanford")

- Query: https://oauth.reddit.com/r/Coldplay/search?q=stanford&sort_by=new&limit=100&restrict_sr=true
- Query: https://oauth.reddit.com/r/Coldplay/search?q=stanford&sort_by=new&limit=100&restrict_sr=true&after=t3_1kwus1m
Submissions query for subreddit_name='Coldplay', query_term='stanford' yielded 122 submissions


In [95]:
comments_res = get_comments("1lhoxch")
len(comments_res)

- Query: https://oauth.reddit.com/comments/1lhoxch?limit=500&sort=old&depth=10
-- Query: https://oauth.reddit.com/api/morechildren?link_id=t3_1lhoxch&limit_children=false&sort=old&api_type=json&children=mz6qy4g,mz6d807,mz5zqk1,mz6cdqz,mz5wfnn,mz69drd,mz69lp2,mz64xcx,mz6e7iq,mz6ejnf,mz728ff,mz6kckn,mz6fw11,mz7jjes,mz5xoyd,mz5wkj9,mz6dq2d,mz5znhy,mz5zz6f,mz63n73,mz673og,mz6oo8i,mz7193w,mz7b9do,mzald9x,mz5ze2p,mz6x6w3,mz6f0dx,mz9dmp2,mz7u315,mz6jtws,mz695is,mz9dt1n,mz69zkl,mz7gnfe,mz7kmph,mz6a4fi,mz6buml,mz61sqq,mz64rw4,mz6ccl9,mz6corx,mz6oz3k,mz6hqxt,mz6qmca,mz84wxq,mz5x90s,mz6k5qc,mz62ero,mz9iha3,mz7w9j4,mz6bf4z,mz94oip,mz6ux30,mz7dade,mz6r8ph,mz6i9e2,mz7jrbt,mza6ttr,mz5xpr5,mz61fm6,mz66hrr,mz6ryas,mz7fwbr,mz6smm5,mz6w37c,mz7i7nu,mz813hd,mz8xeum,mz6er71,mz6zrw4,mz89gxp,mz8dax8,mz62bcf,mz66u1s,mz7jss7,mz688v8,mz6szms,mz5wtts,mz5xhf1,mz6oawt,mzabew1,mz6kdjo,mz6p405,mz6wq9s,mz7ixe4,mz7sjh7,mz80tgm,mz9ayny,mz5whm6,mz61jqn,mz647yh,mz68mb6,mz6iouh,mz6mdlm,mz7znd1,mz67uql,mz6d8js,mz6eefw,mz6jn

7549

In [90]:
extract_post_data(posts_res, "coldplay-stanford.csv")

In [98]:
extract_comment_data(comments_res, "gaming-post-comments.csv")